In [1]:
import os
import tempfile
import pandas as pd
import csv

from lib.export_google_sheets import export_google_sheets


folder_ids=[
        '1IDl3cSc8u5y4Gf13_PBCEZdFFnDv6Ilz', #入力
        '1HhIkzwUMBTfStSCyhSRf1SkbQq21hLnM', #データフォーマット
        '1NrjWTGweLNoUU1n-EH9yhIU5ZBUHOLs1', # R4 version0131に対する査読
        '1FlCwwi71s1BI4uhOf2CefX85byj1Alo_', #各チーム編集用シート
]
DIST_DIR="./raw/sheets/"

try:
    with tempfile.TemporaryDirectory() as temp_dir:
        file_list=export_google_sheets(folder_ids,temp_dir,info_stor_file="./raw/sheets/sheets_info.json")
        for file in file_list:
            book = pd.ExcelFile(file["export_path"])
            dir_name = os.path.join(DIST_DIR,file['name'])
            os.makedirs(dir_name,exist_ok=True)
            for name in book.sheet_names:
                sheet:pd.DataFrame = book.parse(name)
                sheet.to_csv(os.path.join(dir_name,f"{name}.csv"),encoding="utf_8_sig",index=False,)
except:
    import traceback
    traceback.print_exc()

連携編集用 in 2022-03-14T08:57:27.025Z
科学的探究_0131版_査読 in 2022-03-14T08:54:03.115Z
多職種連携能力_0131版_査読 in 2022-03-14T08:53:57.015Z
社会における医療の役割の理解_0131版_査読 in 2022-03-14T08:53:54.788Z
生涯にわたって共に学ぶ姿勢_0131版_査読 in 2022-03-14T08:53:52.161Z
プロフェッショナリズム_0131版_査読 in 2022-03-14T08:53:12.296Z
プロ編集用 in 2022-03-14T08:53:04.467Z
総合編集用 in 2022-03-14T08:52:57.389Z
情報編集用 in 2022-03-14T08:52:53.881Z
技能編集用 in 2022-03-14T08:51:20.484Z
科学編集用 in 2022-03-14T08:50:08.098Z
コミュ編集用 in 2022-03-14T08:49:36.950Z
R4コアカリ-集計情報 in 2022-03-14T08:45:31.116Z
社会編集用 in 2022-03-14T08:40:17.185Z
生涯編集用 in 2022-03-14T08:39:51.174Z
知識編集用 in 2022-03-14T08:31:21.347Z
